# Letzter Stand
- Meeting mit Informatikperson: 7. September 11:00
- Fact-checked output of preprocessing script. 
- re-shaping does not have to be done for non-synchr. input vectors.

Todo:
- Hab ich die Reihenfolge von dimensionality reduction & clustering richtig verstanden?
  - Folien von Yulias Vortrag & Anastasia durchgehen
  - https://tacosconference.github.io/css/2023_style/TalkSlides/YABLOKOVA.pdf
- Je nach above, dimensions von clustering output für PCA anpassen oder andersrum durchführen

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
import fasttext as ft
import numpy as np

- die scikit learn documentation empfiehlt PCA vor k means laufen zu lassen und wird machen das Gegenteil??
- k means sucht nach cluster with equal variance, ist das bei unseren clulstern gegeben?

In [ ]:
# Load data.
forms_full = [
    {"nomn"},
    {"gent"},
    {"datv"},
    {"accs"},
    {"ablt"},
    {"loct"},
    {"gen2"},
    {"plur", "nomn"},
    {"plur", "gent"},
    {"plur", "datv"},
    {"plur", "accs"},
    {"plur", "ablt"},
    {"plur", "loct"},
    {"plur", "gen2"},
]

# For each noun: Lemma, tags (gender, animacy), something, all 14 cases (see above).
list_of_all_forms = np.loadtxt(
    "../../data/list_of_noun_forms_full.csv", dtype=object, delimiter=","
)

# Binary coding of synchretisms. A form at index i,j is not syncretic if matrix_of_syncr[i][j]==2**j.
matrix_of_syncr = np.loadtxt(
    "../../data/matrix_of_syncr.csv", dtype=float, delimiter=","
).astype(int)

# List of cases without lemma and tags.
list_forms = list_of_all_forms[:, 4:]

# Load fasttext model.
model = ft.load_model(".l./../data/final-model.bin")

# Get all the vectors in the array.
# Gets the vector for all word lemmas from list_forms.
vectors_all_nouns = [[model.get_word_vector(x) for x in line] for line in list_forms]

In [ ]:
d = np.load("../../data/nonsynchr_casevectors.npy")

# Removing synchronous forms

`list_forms` and `matrix of synchr.` have the same length so presumably the same structure.
> 11515 lines and 14 dims per line 

```python
matrix_of_syncr[0]
array([   1,   74,    4,   74,   16,   32,   74,  128, 9472,  512, 9472,
       2048, 4096, 9472])
```

```python
list_forms[0]
array(['человек', 'человека', 'человеку', 'человека', 'человеком',
       'человеке', 'человека', 'люди', 'людей', 'людям', 'людей',
       'людьми', 'людях', 'людей'], dtype=object)
```

In [ ]:
# Get case vectors for all non-synchretic cases.
nonsynchr_casevectors = []
for form in range(0, len(list_forms)):
    for case in range(0, 14):
        if (
            matrix_of_syncr[form][case] == 2**case
        ):  # Get word vector if case is not synchretic.
            vector = model.get_word_vector(list_forms[form][case])
            nonsynchr_casevectors.append(vector)
print(len(nonsynchr_casevectors), len(list_forms))  # 68687 11515
# np.save('../data/nonsynchr_casevectors.npy', nonsynchr_casevectors)

In [ ]:
d = np.load("../../data/nonsynchr_casevectors.npy")
d.shape  # (68687, 300)
# model.get_word_vector("человек").shape # (300,)

In [ ]:
len(vectors_all_nouns[0])

### Clustering

In [ ]:
def cluster_kmeans(n_clusters, input):
    """Cluster the input into n_clusters using kmeans.

    Parameters
    ----------
    n_clusters: int
        Number of clusters to cluster the input into.
    input: array-like
        Input to cluster.

    Returns
    -------
    labels: array-like
        Cluster labels for each element in the input.
    centers: array-like
        Cluster centers.
    """
    x = np.ascontiguousarray(input)
    # nsamples, nx, ny = x.shape
    # x = x.reshape((nsamples, nx * ny))
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(x)
    return kmeans.labels_, kmeans.cluster_centers_

In [ ]:
# labels = cluster_kmeans(8, d)  # Whats a good number of clusters?
# d = np.load("../../data/nonsynchr_casevectors.npy")
kmeans = KMeans(n_clusters=9, random_state=0, n_init="auto").fit(d)
# explain the random state parameter in kmeans
kmeans_silhouette = silhouette_score(d, kmeans.labels_).round(2)
kmeans_silhouette

# Dim reduction

In [ ]:
# The PCA/TSNE n_dimensions that anastasia used:
# pca: 2/50 dimensions
# tsne: 2/3 dimensions


def run_pca(vectors, pca_dims):
    """Run dimensionality reduction using Principal Component Analysis.

    Parameters
    ----------
    vectors: array-like
        Vectors to reduce.
    pca_dims: int
        Number of dimensions to reduce to.

    Returns
    -------
    results: array-like
        Reduced vectors.
    """
    pca = PCA(n_components=pca_dims)
    results = pca.fit_transform(vectors)
    return results


def run_pca_tsne(vectors, pca_dims, tsne_dims):
    """Run dimensionality reduction using Principal Component Analysis and t-SNE in that order.

    Parameters
    ----------
    vectors: array-like
        Vectors to reduce.
    pca_dims: int
        Number of dimensions to reduce to using PCA.
    tsne_dims: int
        Number of dimensions to reduce to using t-SNE.

    Returns
    -------
    tsne_result: array-like
        Reduced vectors.
    """
    pca = PCA(n_components=pca_dims)
    tsne = TSNE(n_components=tsne_dims)
    pca_result = pca.fit_transform(vectors)
    tsne_result = tsne.fit_transform(pca_result)
    return tsne_result

In [ ]:
clusters = run_pca(, 2)

In [ ]:
clusters